In [ ]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import glob
import os
from obspy import read, UTCDateTime, read_inventory
from obspy.signal import PPSD
import warnings
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use("ggplot")

from msnoise.api import *

In [ ]:
# connect to the database
db = connect()
params = get_params(db)

In [ ]:
! msnoise info -j

compute the mwcs & dtt

In [ ]:
! msnoise -q -t 5 cc dvv compute_mwcs

In [ ]:
! msnoise -q -t 5 cc dvv compute_dtt

In [ ]:
! msnoise cc dvv compute_dvv

plot the final results

examples:

- msnoise cc dvv plot dvv -f 1
- msnoise cc dvv plot dvv -f 2 -m 2

In [ ]:
from msnoise.plots.dvv import main as plot_dvv
for filterid in [1, 2]:
    plot_dvv(filterid=filterid, show=False)
    for ax in plt.gcf().axes:
        plt.sca(ax)
        plt.axvline(datetime.datetime(2014,6,21,),c='r', ls='--')
    plt.show()

Using the coda window dynamically from the interstation distance

In [ ]:
update_config(db, name="dtt_lag", value="dynamic")

In [ ]:
! msnoise reset DTT --all
! msnoise -t 5 cc dvv compute_dtt
! msnoise cc dvv compute_dvv

In [ ]:
from msnoise.plots.dvv import main as plot_dvv
for filterid in [1, 2]:
    plot_dvv(filterid=filterid, show=False)
    for ax in plt.gcf().axes:
        plt.sca(ax)
        plt.axvline(datetime.datetime(2014,6,21,),c='r', ls='--')
    plt.show()

Or plotting individual pairs:

In [ ]:
filters = get_filters(db)
comp = "ZZ"

for mov_stack in params.mov_stack:
    fig, axes = plt.subplots(2,2, figsize=(10,10))
    plt.suptitle(mov_stack)
    for i, filter in enumerate(filters):
        filterid = filter.ref
        col = i % 2
        for sta1, sta2 in get_station_pairs(db):
            row = i // 2
            plt.sca(axes[row,col])
            for loc1 in sta1.locs():
                s1 = "%s.%s.%s" % (sta1.net, sta1.sta, loc1)
                for loc2 in sta2.locs():
                    s2 = "%s.%s.%s" % (sta2.net, sta2.sta, loc2)
                    if s1 == s2:
                        continue
                    dvv = compute_dvv(db, filterid, mov_stack, components=comp, params=params, pairs=[(s1,s2),])
                    dvv.m *= -100
                    dvv.m.plot(label="%s:%s"%(s1,s2), ax=axes[row,col])
                    plt.title("Filter %i (%.2f - %.2f Hz) - %s" % (filterid, filter.low, filter.high, comp))
            plt.axvline(datetime.date(2014,6,21), c='r', ls='--')
            plt.legend()
            plt.xlabel("")
            plt.ylabel("dv/v (%)")
    plt.tight_layout()
    plt.show()